# Projeto 1 - Ciência dos Dados

Nome: _Marco Tulio Masselli Rainho Teixeira_

Nome: _Talissa Gonçalves Albertini_

___
## Classificador automático de sentimento


Este classificador automático de sentimento tem o objetivo de promover uma otimização no processo de interpretação de feed back de qualquer produto no mercado, identificando características que os usuários aprovam ou reprovam, e no ambiente do Twitter, selecionando comentários que tenham uma maior probabilidade de serem relevantes à equipe que os analizará.

### 1. Preparação do ambiente do Jupyter

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
from string import punctuation
import nltk 
from nltk.stem import RSLPStemmer
nltk.download('rslp')

[nltk_data] Downloading package rslp to C:\Users\Marco
[nltk_data]     Tulio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True